In [16]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [17]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

In [18]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [19]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 7
Component distribution: {'min': 1, 'p5': 1, 'max': 7, 'p999': 7, 'p99': 7, 'p1': 1, 'p10': 1, 'p90': 7, 'p50': 2, 'p25': 2, 'p75': 4, 'p95': 7, 'mean': 2.857142857142857}


In [20]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                             28
nodePropertiesWritten                                                   20
ranLevels                                                                1
didConverge                                                           True
nodeCount                                                               20
communityCount                                                           8
communityDistribution    {'min': 1, 'p5': 1, 'max': 4, 'p999': 4, 'p99'...
modularity                                                        0.713333
modularities                                          [0.7133333333333333]
postProcessingMillis                                                    16
preProcessingMillis                                                      0
computeMillis                                                           34
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [21]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 20}]

In [22]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [23]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,8,2.0,2.0,3.25,4.0,4.0,4


In [24]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info

[{'communityId': '0-1',
  'nodes': [{'id': '金融機構',
    'description': '核印和扣款時間需參照各金融機構的相關規定。',
    'type': 'Organization'},
   {'id': '保戶', 'description': '保戶指的是與保險公司簽約的個人或團體。', 'type': 'Person'},
   {'id': '首期保險費',
    'description': '首期保險費是保戶首次繳交的保險費用，採用金融機構轉帳作業。',
    'type': 'Concept'},
   {'id': '保險', 'description': '轉帳扣款作業需經核印成功且同意承保後始得進行。', 'type': '概念'}],
  'rels': [{'start': '金融機構',
    'description': None,
    'type': '申請',
    'end': '首期保險費'},
   {'start': '金融機構', 'description': None, 'type': '轉帳繳交', 'end': '首期保險費'},
   {'start': '保險',
    'description': '核印和扣款時間需參照各金融機構的相關規定。',
    'type': '依照',
    'end': '金融機構'},
   {'start': '首期保險費', 'description': '首期保險費由保戶繳交。', 'type': '繳交', 'end': '保戶'},
   {'start': '首期保險費',
    'description': '首期保險費由保戶約定於本公司簽約之金融機構所開立之活期帳戶扣款。',
    'type': '轉帳',
    'end': '金融機構'}]},
 {'communityId': '0-3',
  'nodes': [{'id': '轉帳繳交', 'description': None, 'type': None},
   {'id': '申請', 'description': None, 'type': None}],
  'rels': [{'start': '申請', '

In [25]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [10]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
model_name = 'qwen2:72b-instruct-q8_0'
llm = OllamaFunctions(model=model_name)
llm

/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 0.4.0. An updated version of the class exists in the langchain-ollama package and should be used instead. To use it run `pip install -U langchain-ollama` and import as `from langchain_ollama import ChatOllama`.
  warn_deprecated(


OllamaFunctions(model='qwen2:72b-instruct-q8_0')

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """請根據提供同社區的資訊包含 nodes 與 relationships,
產生自然語言的摘要 同社區的資訊
請確保使用繁體中文回應:
{community_info}

Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input triples, generate the information summary. No pre-amble.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [27]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '0-1',
 'summary': '在這個社區中，有四個主要節點和它們之間的關係：\n\n1. **金融機構** (Organization)\n   - 描述: 核印和扣款時間需參照各金融機構的相關規定。\n   - 申請首期保險費。\n   - 透過轉帳繳交首期保險費。\n\n2. **保戶** (Person)\n   - 描述: 保戶指的是與保險公司簽約的個人或團體。\n   - 繳交首期保險費。\n\n3. **首期保險費** (Concept)\n   - 描述: 首期保險費是保戶首次繳交的保險費用，採用金融機構轉帳作業。\n   - 首期保險費由保戶繳交。\n   - 透過轉帳從金融機構扣款。\n\n4. **保險** (概念)\n   - 描述: 轉帳扣款作業需經核印成功且同意承保後始得進行。\n   - 依照金融機構的相關規定進行。\n\n總結來說，保戶與保險公司簽約後，首期保險費由保戶繳交，並透過金融機構進行轉帳作業。整個過程需要依照金融機構的核印和扣款時間規定。'}

In [28]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def process_community_with_llm(community_info, max_retry=0):
    '''
    params:
        community_info: [ { 
                            'communityId': str, 'nodes': [{'id': str, 'description': str|None, 'type': str}, ...], 
                            'rels': [{'start': str, 'description': str|None, 'type': str, 'end': 'str}, ...]
                          },
                          ... ]
        max_retry: 最多嘗試次數, 假設為2, 則最多遞迴執行 2+1=3次
    '''
    summaries = []
    faild_communities = []
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_community, community): community for community in community_info}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
            try:
                summaries.append(future.result())
            except Exception as e:
                community = futures[future]
                print(f'process community faild!:{community}, error:\n{e}')
                faild_communities.append(community)
    if max_retry > 0:
        summaries.extend(process_community_with_llm(faild_communities, max_retry=max_retry-1))
    return summaries
summaries = process_community_with_llm(community_info, max_retry=2)

Processing communities: 100%|██████████| 7/7 [00:05<00:00,  1.28it/s]


In [29]:
summaries

[{'community': '0-3',
  'summary': '在這個社區中，有兩個主要的節點：一個是「申請」，另一個是「轉帳繳交」。這兩個節點之間存在一個關係，表示「申請」包含了「轉帳繳交」的過程。'},
 {'community': '0-18',
  'summary': '保單是一個概念，其生效日認定依照「生效日期之認定原則」辦理。這兩者之間的關係是保單依照生效日期之認定原則來確定生效日期。'},
 {'community': '0-13',
  'summary': '在同一個社區中，有四個節點和三個關係。四個節點分別是「土地銀行」、「第一銀行」、「合作金庫」和「數位帳戶」。其中「土地銀行」、「第一銀行」和「合作金庫」都是組織類型的節點，且這三個節點都有無法授權扣款的描述。「數位帳戶」則是概念類型的節點，其描述為需要至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。\n\n在關係部分，「數位帳戶」與「土地銀行」、「第一銀行」以及「合作金庫」之間均存在無法授權扣款的關係。具體來說，數位帳戶無法在「土地銀行」、「第一銀行」和「合作金庫」授權扣款。'},
 {'community': '0-1',
  'summary': '同社區的資訊摘要如下：\n\n在這個社區中，有若干個節點和關係。節點包括：\n1. 金融機構：一個組織，其核印和扣款時間需參照各金融機構的相關規定。\n2. 保戶：指的是與保險公司簽約的個人或團體。\n3. 首期保險費：保戶首次繳交的保險費用，通過金融機構轉帳作業。\n4. 保險：轉帳扣款作業需經核印成功且同意承保後始得進行。\n\n這些節點之間的關係如下：\n1. 金融機構會申請首期保險費。\n2. 金融機構會轉帳繳交首期保險費。\n3. 保險依照金融機構的相關規定，核印和扣款時間需參照各金融機構的相關規定。\n4. 首期保險費由保戶繳交。\n5. 首期保險費由保戶約定於本公司簽約之金融機構所開立之活期帳戶扣款。'},
 {'community': '0-7',
  'summary': '在同一社區中，有兩個主要資訊節點。第一個節點是"新契約投保時授權書"，它是一份文件，描述為"不得為空白授權，應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書"。第二個節點是"自動轉帳及信用卡付款授權書作業規範"，這是一項規範，描述為"加

In [30]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]